This script is used to perform data processing on year2 data and ingest the features and the newly engineered features to the existing feature groups

## Required Imports

In [3]:
import sagemaker
import sys

import boto3
import pandas as pd
import numpy as np
import io
import time
from time import gmtime, strftime, sleep
import datetime

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

## Loading the Data from S3

In [4]:
############################################################
year = "2year"
# Loads the 5 raw .arff files into pandas dataframes
def load_dataframes():
    data_loc = f's3://sagemaker-featurestore-pov/bankruptcy data/{year}.csv'
    return pd.read_csv(data_loc)

# dataframes is the list of pandas dataframes for the 5 year datafiles.  
dataframe = load_dataframes()


In [5]:
dataframe

,1,2,3,4,5,6,7,8,9,10,...,56,57,58,59,60,61,62,63,64,65
0,0.202350,0.46500,0.240380,1.51710,-14.54700,0.510690,0.253660,0.918160,1.15190,0.426950,...,0.131840,0.473950,0.86816,0.000240,8.5487,5.16550,107.740,3.38790,5.3440,0
1,0.030073,0.59563,0.186680,1.33820,-37.85900,-0.000319,0.041670,0.678900,0.32356,0.404370,...,0.121460,0.074369,0.87235,0.000000,1.5264,0.63305,622.660,0.58619,1.2381,0
2,0.257860,0.29949,0.665190,3.22110,71.79900,0.000000,0.318770,2.332000,1.67620,0.698410,...,0.164990,0.369210,0.81614,0.000000,4.3325,3.19850,65.215,5.59690,47.4660,0
3,0.227160,0.67850,0.042784,1.08280,-88.21200,0.000000,0.285050,0.473840,1.32410,0.321500,...,0.293580,0.706570,0.78617,0.484560,5.2309,5.06750,142.460,2.56210,3.0066,0
4,0.085443,0.38039,0.359230,1.94440,21.73100,0.187900,0.108230,1.371400,1.11260,0.521670,...,0.101240,0.163790,0.89876,0.000000,5.7035,4.00200,89.058,4.09840,5.9874,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10168,0.029970,0.66806,0.066243,1.11030,-105.55000,0.029970,0.038888,0.482740,1.02920,0.322500,...,0.028377,0.092931,0.97162,0.209820,3.0914,3.94560,192.220,1.89880,3.4199,1
10169,0.012843,0.49306,-0.160620,0.61898,-24.80100,0.012843,0.012843,0.905900,1.01450,0.446660,...,0.014247,0.028752,0.98575,0.160090,48.6660,63.75200,40.071,9.10870,5.1956,1
10170,0.015092,0.55759,-0.284600,0.48599,-85.57100,0.015092,0.009826,0.694880,1.00600,0.387460,...,0.005971,0.038950,0.99403,0.010091,15.0530,11.96400,114.250,3.19480,2.4201,1
10171,-0.002554,0.47076,0.424010,1.90070,0.95483,-0.002554,0.001785,1.114400,0.99293,0.524640,...,-0.007122,-0.004869,1.00710,0.000000,6.4289,5.70250,64.291,5.67730,25.3990,1


## Data Preprocessing and New feature generation

In [6]:
dataframe = dataframe.fillna(dataframe.mean())

In [7]:
# Adding columns with Id and Year for Record Identifier and Event Time
# Add event_time timestamp. This is a point in time when a new event occurs that corresponds to the creation or update of a record in a feature group
### timestamp = str(datetime.datetime.now())

dataframe['id'] = np.arange(len(dataframe))

current_time_sec = int(round(time.time()))
dataframe['event_time'] = current_time_sec
dataframe['year'] = year
dataframe = dataframe.astype({'event_time': 'float64'})
dataframe.rename({'65': 'labels'}, axis=1, inplace=True)

new_columns = ['id', 'event_time', 'year', 'labels']

In [8]:
def new_feature_addition(data):
            feature1 = data['6'] / data['7']
            feature2 = data['55'] / data['28']
            feature3 = (data['55'] * data['1']) / data['3']
            
            year = data['year']
            Id = data['id']
            EventTime = data['event_time']

            new_df = pd.DataFrame([Id, EventTime, year, feature1, feature2, feature3])
            new_df = new_df.T
            new_df.columns = ['id', 'event_time', 'year', 'attr66', 'attr67', 'attr68']

            new_df.replace([np.inf, -np.inf], np.nan, inplace=True)
            new_df = new_df.fillna(dataframe.mean())
            new_df = new_df.astype({'id':'int'})

            return (new_df)

In [9]:
df_new = new_feature_addition(dataframe)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  from ipykernel import kernelapp as app


In [10]:
schema = ['7', '12', '15', '27', '41', '55']

for i in range(len(schema)):
    dataframe.rename({schema[i]: 'attr' + schema[i]}, axis=1, inplace=True)
    schema[i] = 'attr' + schema[i]
    i += i
    

schema.extend(new_columns)

In [11]:
df = dataframe[schema]

In [12]:
df.head()

,attr7,attr12,attr15,attr27,attr41,attr55,id,event_time,year,labels
0,0.25366,0.545610,603.20,1.6345,0.060856,242500.0,0,1.676985e+09,2year,0
1,0.04167,0.075493,4613.90,79.7520,0.416710,2179.5,1,1.676985e+09,2year,0
2,0.31877,1.064400,330.29,52.8860,0.031507,3668.6,2,1.676985e+09,2year,0
3,0.28505,0.551560,820.52,3.9521,0.057077,1273.8,3,1.676985e+09,2year,0
4,0.10823,0.284520,997.01,1.3442,0.057007,18209.0,4,1.676985e+09,2year,0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10173 entries, 0 to 10172
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   attr7       10173 non-null  float64
 1   attr12      10173 non-null  float64
 2   attr15      10173 non-null  float64
 3   attr27      10173 non-null  float64
 4   attr41      10173 non-null  float64
 5   attr55      10173 non-null  float64
 6   id          10173 non-null  int64  
 7   event_time  10173 non-null  float64
 8   year        10173 non-null  object 
 9   labels      10173 non-null  int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 794.9+ KB


In [14]:
df_new.head()

,id,event_time,year,attr66,attr67,attr68
0,0,1.676985e+09,2year,2.013286,297330.766684,204134.599384
1,1,1.676985e+09,2year,-0.007647,3051.153545,351.104047
2,2,1.676985e+09,2year,0.000000,194.765343,1422.127807
3,3,1.676985e+09,2year,0.000000,13112.222840,6763.192034
4,4,1.676985e+09,2year,1.736118,13197.796622,4331.017975


## Ingest data to the two existing Feature Group

In [15]:
feature_group_name = 'bankruptcy-fg-2023-02-21-12-55-30'

In [16]:
from sagemaker.feature_store.feature_group import FeatureGroup

feature_group1 = FeatureGroup(name=feature_group_name, sagemaker_session=sess)

In [17]:
feature_group1.ingest(data_frame=df, max_workers=3, max_processes = 1, wait=True)

IngestionManagerPandas(feature_group_name='bankruptcy-fg-2023-02-21-12-55-30', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7fd50fd43f50>, sagemaker_session=<sagemaker.session.Session object at 0x7fd510a22810>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7fd5126b2b90>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

In [18]:
feature_group_name = 'bankruptcy-new-features-2023-02-21-12-56-19'

In [19]:
feature_group2 = FeatureGroup(name=feature_group_name, sagemaker_session=sess)

In [20]:
feature_group2.ingest(data_frame=df_new, max_workers=2, wait=True)

IngestionManagerPandas(feature_group_name='bankruptcy-new-features-2023-02-21-12-56-19', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7fd50fd43f50>, sagemaker_session=<sagemaker.session.Session object at 0x7fd510a22810>, max_workers=2, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7fd5126bcf90>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

## Check the two feature groups

In [21]:
query = feature_group1.athena_query()
table_name = query.table_name
print(table_name)
query_string = ('SELECT * FROM "%s"' % table_name)
print('Running ' + query_string)

query.run(query_string=query_string,
          output_location=f's3://sagemaker-featurestore-pov/feature-group-storage')
query.wait()
dataset1 = query.as_dataframe()

bankruptcy-fg-2023-02-21-12-55-30-1676984149
Running SELECT * FROM "bankruptcy-fg-2023-02-21-12-55-30-1676984149"


In [22]:
dataset1

,attr7,attr12,attr15,attr27,attr41,attr55,id,event_time,year,labels,write_time,api_invocation_time,is_deleted
0,0.118290,12.530000,22.182,10.451000,0.001875,1920.100,6788,1.676985e+09,2year,0,2023-02-21 13:10:42.172,2023-02-21 13:05:48.000,False
1,-0.042804,-0.084323,8649.200,-4.348100,0.974250,374.640,3400,1.676985e+09,2year,0,2023-02-21 13:10:42.172,2023-02-21 13:05:48.000,False
2,0.354110,0.501290,672.150,38.298000,0.059901,53.313,32,1.676985e+09,2year,0,2023-02-21 13:10:42.172,2023-02-21 13:05:48.000,False
3,0.159320,0.205570,1750.700,0.906130,0.116630,2577.600,41,1.676985e+09,2year,0,2023-02-21 13:10:42.172,2023-02-21 13:05:48.000,False
4,0.190190,1.354600,243.950,10.540000,0.021991,189.500,51,1.676985e+09,2year,0,2023-02-21 13:10:42.172,2023-02-21 13:05:48.000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17195,0.135900,0.276230,1516.600,17.876000,0.134360,1212.800,3121,1.676984e+09,1year,0,2023-02-21 13:02:22.451,2023-02-21 12:57:53.000,False
17196,0.043024,0.077069,2910.200,3.983600,0.247020,328.000,6818,1.676984e+09,1year,1,2023-02-21 13:02:22.451,2023-02-21 12:57:54.000,False
17197,0.085709,0.141330,3107.700,1.461400,0.175320,7306.400,3246,1.676984e+09,1year,0,2023-02-21 13:02:22.451,2023-02-21 12:57:55.000,False
17198,-0.014404,-0.022886,7554.800,1321.989035,0.216990,-6440.000,6926,1.676984e+09,1year,1,2023-02-21 13:02:22.451,2023-02-21 12:57:55.000,False


In [23]:
query = feature_group2.athena_query()
table_name = query.table_name
print(table_name)
query_string = ('SELECT * FROM "%s"' % table_name)
print('Running ' + query_string)

query.run(query_string=query_string,
          output_location=f's3://sagemaker-featurestore-pov/feature-group-storage')
query.wait()
dataset2 = query.as_dataframe()

bankruptcy-new-features-2023-02-21-12-56-19-1676984204
Running SELECT * FROM "bankruptcy-new-features-2023-02-21-12-56-19-1676984204"


In [24]:
dataset2

,id,event_time,year,attr66,attr67,attr68,write_time,api_invocation_time,is_deleted
0,39,1.676985e+09,2year,6.152800,5449.275952,54.089863,2023-02-21 13:11:23.737,2023-02-21 13:06:32.000,False
1,45,1.676985e+09,2year,0.000000,637.647901,2107.412381,2023-02-21 13:11:23.737,2023-02-21 13:06:33.000,False
2,5220,1.676985e+09,2year,0.000000,2899.367302,58.431602,2023-02-21 13:11:23.737,2023-02-21 13:06:33.000,False
3,123,1.676985e+09,2year,0.852538,66472.768289,13264.567496,2023-02-21 13:11:23.737,2023-02-21 13:06:33.000,False
4,124,1.676985e+09,2year,0.000000,3423.351184,455.433124,2023-02-21 13:11:23.737,2023-02-21 13:06:34.000,False
...,...,...,...,...,...,...,...,...,...
17195,6871,1.676984e+09,1year,1.000000,16158.353899,4843.338099,2023-02-21 13:03:32.964,2023-02-21 12:58:46.000,False
17196,6878,1.676984e+09,1year,0.974855,4.830013,-265.301746,2023-02-21 13:03:32.964,2023-02-21 12:58:46.000,False
17197,6916,1.676984e+09,1year,1.000000,9575.125161,-7349.054598,2023-02-21 13:03:32.964,2023-02-21 12:58:47.000,False
17198,3492,1.676984e+09,1year,0.000000,2082.841674,-12.031861,2023-02-21 13:03:32.964,2023-02-21 12:58:48.000,False


In [24]:
dataset1.value_counts('event_time')

event_time
1.676037e+09    10173
1.676036e+09     7027
dtype: int64

In [27]:
dataset2.value_counts('event_time')

event_time
1.676037e+09    10173
1.676036e+09     7027
dtype: int64